# IOMEval
> Streamline evaluation reports into structured knowledge maps with LLMs


[![CI](https://github.com/franckalbinet/iomeval/actions/workflows/test.yaml/badge.svg)](https://github.com/franckalbinet/iomeval/actions/workflows/test.yaml)
[![PyPI](https://img.shields.io/pypi/v/iomeval)](https://pypi.org/project/iomeval/)
[![License](https://img.shields.io/badge/license-Apache%202.0-blue.svg)](LICENSE)

`iomeval` streamlines the mapping of [IOM](https://www.iom.int) evaluation reports against strategic frameworks like the [Strategic Results Framework (SRF)](https://srf.iom.int/) and [Global Compact for Migration (GCM)](https://www.un.org/en/development/desa/population/migration/generalassembly/docs/globalcompact/A_RES_73_195.pdf). It uses LLMs to process PDF reports, extract key sections, and tag/map them to framework components, turning dispersed, untagged evaluation documents into structured, searchable knowledge maps.

## Why This Matters

UN agencies produce extensive evaluation reports and other public documents. For IOM, this body of knowledge is extensive and variegated, but putting it to practical use becomes more challenging as volume increases, particularly when documentation is stored across different repositories with no single index available.

**The Challenge for IOM**

IOM's evaluation production is highly decentralized, with reports stored across multiple repositories (the [IOM Evaluation Repository](https://evaluation.iom.int/evaluation-search-pdf), IOM Library, IOM Protection Platform). Quality varies greatly: quality control processes are not applied uniformly, and variation also reflects the inherent subjectivity in evaluation approaches and interpretations. Reports also vary significantly in structure: some follow common formats with executive summaries, findings, and recommendations, while others have different structures entirely. This inconsistency makes systematic mapping challenging.

Critically, existing metadata doesn't indicate which elements of IOM's strategic frameworks (the [Strategic Results Framework (SRF)](https://srf.iom.int/) or the [Global Compact for Migration (GCM)](https://www.un.org/en/development/desa/population/migration/generalassembly/docs/globalcompact/A_RES_73_195.pdf)) each report addresses. This is a major gap that limits the ability to connect evaluation evidence with key strategic frameworks.

**Evidence Maps as a Solution**

Evidence Maps display the extent and nature of research and evaluation available on a subject. Following the [2025 UNEG Eval Week](https://www.unevaluation.org/events/uneg-evaluation-week-2025), four primary use cases emerged: guiding future evidence generation, informing policy decisions, knowledge management, and enhancing collaboration. The maps created by `iomeval` serve primarily as **knowledge management tools**—structured repositories that make identifying relevant sources easier by organizing them against strategic framework components.

**What This Enables**

By tagging reports against SRF outputs, enablers, cross-cutting priorities, and GCM objectives, these maps help answer questions like: Which framework elements are well-covered by existing evaluations? Where are the knowledge gaps that should prioritize future evaluation work? Which themes have enough evidence for a dedicated synthesis report?

## Key Features

- **Automated PDF Processing**: Download and OCR evaluation reports with proper heading hierarchy
- **Intelligent Section Extraction**: LLM-powered extraction of executive summaries, findings, conclusions, and recommendations
- **Strategic Framework Mapping**: Map report content to IOM's SRF Enablers, Cross-Cutting Priorities, GCM Objectives, and SRF Outputs
- **Checkpoint/Resume**: Built-in state persistence - interrupt and resume long-running pipelines
- **Granular Control**: Use the full pipeline or individual components as needed

## Installation

Install from PyPI:
```sh
pip install iomeval
```

Or install the latest development version from GitHub:
```sh
pip install git+https://github.com/franckalbinet/iomeval.git
```

## Configuration

### Core Dependencies

iomeval relies on two key libraries:

- **[mistocr](https://fr.anckalbi.net/mistocr)**: Powers the PDF-to-markdown conversion with intelligent OCR and heading hierarchy detection
- **[lisette](https://lisette.answer.ai)**: A thin wrapper around [litellm](https://www.litellm.ai/) that provides access to all major LLM providers. By default, iomeval uses Anthropic models (Haiku for debugging, Sonnet for production)

### API Keys

`iomeval` automatically loads API keys on import. You have two options:

**Option 1: Environment variables** (recommended for production)
```sh
export ANTHROPIC_API_KEY='your-key-here'
export MISTRAL_API_KEY='your-key-here'
```

**Option 2: .env file** (convenient for development)

Create a `.env` file in your project root:

```
ANTHROPIC_API_KEY=your-key-here
MISTRAL_API_KEY=your-key-here
```

Since lisette supports all major LLM providers via litellm, you can configure other providers (OpenAI, Google, etc.) by setting their respective API keys using either method.

## Quick Start

First, prepare your evaluations data. Export evaluations from [IOM's evaluation repository](https://evaluation.iom.int/evaluation-search-pdf) as CSV, then convert to JSON:

```python
from iomeval.readers import IOMRepoReader

reader = IOMRepoReader('evaluation-search-export.csv')
reader.to_json('evaluations.json')
```

Now process an evaluation report end-to-end:

```python
from iomeval.readers import load_evals
from iomeval.pipeline import run_pipeline

evals = load_evals('evaluations.json')
url = "https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Abridged%20Evaluation%20Report_%20Final_Olta%20NDOJA.pdf"

report = await run_pipeline(url, evals, 
                            pdf_dst='data/pdfs', 
                            md_dst='data/md', 
                            results_path='data/results', 
                            ocr_kwargs=dict(add_img_desc=False), 
                            model='claude-haiku-4-5')
report
```

The pipeline runs 7 steps: 

**download → OCR → extract → map SRF Enablers → map SRF Cross-cutting Priorities → map GCM Objectives → map SRF Outputs** 

Progress is displayed as each step completes, and state is automatically saved after each stage for checkpoint/resume capability.

## Detailed Workflow

For more control over individual pipeline stages, see the module documentation:

- **Loading evaluation metadata**: See [readers](https://fr.anckalbi.net/iomeval/readers.html) for working with IOM evaluation data
- **Downloading and OCR**: See [downloaders](https://fr.anckalbi.net/iomeval/downloaders.html) and [core](https://fr.anckalbi.net/iomeval/core.html) for PDF processing
- **Section extraction**: See [extract](https://fr.anckalbi.net/iomeval/extract.html) for extracting executive summaries, findings, conclusions, and recommendations
- **Framework mapping**: See [mapper](https://fr.anckalbi.net/iomeval/mapper.html) for mapping to SRF enablers, cross-cutting priorities, GCM objectives, and SRF outputs
- **Pipeline control**: See [pipeline](https://fr.anckalbi.net/iomeval/pipeline.html) for granular control over the full pipeline and checkpoint/resume functionality

## Development

iomeval is built with [nbdev](https://nbdev.fast.ai/), which means the entire library is developed in Jupyter notebooks. The notebooks serve as both documentation and source code.

### Setup for development

```sh
git clone https://github.com/franckalbinet/iomeval.git
cd iomeval
pip install -e '.[dev]'
```

### Key nbdev commands

```sh
nbdev_test          # Run tests in notebooks
nbdev_export        # Export notebooks to Python modules
nbdev_preview       # Preview documentation site
nbdev_prepare       # Export, test, and clean notebooks (run before committing)
```

### Workflow

1. Make changes in the `.ipynb` notebook files
2. Run `nbdev_prepare` to export code and run tests
3. Commit both notebooks and exported Python files
4. Documentation is automatically generated from the notebooks

Learn more about nbdev's literate programming approach in the [nbdev documentation](https://nbdev.fast.ai/).

### Contributing

Contributions are welcome! Please:
- Follow the existing notebook structure
- Run `nbdev_prepare` before submitting PRs

After having run `pip install -U` how to guarantee we have the latest version of my package. In Vscode, I have update a package but still using the previous version. Is there somethign related to cache, ...?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

This is a common frustration! Even after `pip install -U packagename`, you might still be using the old version. Here are the main culprits:

**1. Python kernel/interpreter hasn't restarted**
   - In VSCode, your Python interpreter is still running with the old version loaded in memory
   - Solution: Restart your Python kernel (if using Jupyter) or restart your Python interpreter

**2. Import cache**
   - Python caches imported modules in `sys.modules`
   - Solution: If you can't restart, try:
     ```python
     import sys
     if 'iomeval' in sys.modules:
         del sys.modules['iomeval']
     import iomeval
     ```
   - But honestly, **restarting is cleaner and more reliable**

**3. Verify what's actually installed**
   ```python
   import iomeval
   print(iomeval.__version__)  # Check version
   print(iomeval.__file__)     # Check where it's loading from
   ```

**4. Multiple Python environments**
   - You might have updated the package in one environment but VSCode is using another
   - Check which Python VSCode is using (bottom right corner in VSCode)
   - Verify: `!which python` or `!where python` (Windows)

**5. Development install (`pip install -e`)**
   - If you installed with `-e` (editable mode), pip won't update it
   - You need to uninstall first: `pip uninstall iomeval`, then reinstall

What's your situation? Are you developing iomeval yourself (using `-e` install) or just using it as a regular package?

How to put the code below:

```
from pathlib import Path
from iomeval.readers import IOMRepoReader

data_dir = Path('data')
evl_repo = data_dir / 'evaluation-search-export-12_10_2025--18_07_30.csv'
reader = IOMRepoReader(evl_repo)
reader.to_json(data_dir/'evaluations.json')

from iomeval.readers import load_evals
from iomeval.pipeline import run_pipeline

evals = load_evals(data_dir/'evaluations.json')

url = "https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Abridged%20Evaluation%20Report_%20Final_Olta%20NDOJA.pdf"

report = await run_pipeline(url, evals, 
                            pdf_dst='data/pdfs', 
                            md_dst='data/md', 
                            results_path='data/results', 
                            ocr_kwargs=dict(add_img_desc=False), 
                            model='claude-haiku-4-5')
```

in a python file I can run (considereing the await)

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To run code with `await` in a regular Python file, you have a few options:

**Option 1: Wrap in an async function and use `asyncio.run()`**

```python
import asyncio
from pathlib import Path
from iomeval.readers import IOMRepoReader, load_evals
from iomeval.pipeline import run_pipeline

async def main():
    data_dir = Path('data')
    evl_repo = data_dir / 'evaluation-search-export-12_10_2025--18_07_30.csv'
    reader = IOMRepoReader(evl_repo)
    reader.to_json(data_dir/'evaluations.json')
    
    evals = load_evals(data_dir/'evaluations.json')
    url = "https://evaluation.iom.int/sites/g/files/tmzbdl151/files/docs/resources/Abridged%20Evaluation%20Report_%20Final_Olta%20NDOJA.pdf"
    
    report = await run_pipeline(url, evals, 
                                pdf_dst='data/pdfs', 
                                md_dst='data/md', 
                                results_path='data/results', 
                                ocr_kwargs=dict(add_img_desc=False), 
                                model='claude-haiku-4-5')
    return report

if __name__ == '__main__':
    report = asyncio.run(main())
    print(report)
```

**Option 2: Use `nest_asyncio`** (if running in Jupyter/IPython-like environments)

This is simpler but only needed if you get event loop errors.

Which approach would you like to use? Are you running this as a standalone script or in a notebook environment?

In [ ]:
!ls -R ../../iomeval/iomeval

../../iomeval/iomeval:
__init__.py  _modidx.py  downloaders.py  files	    pipeline.py  themes.py
__pycache__  core.py	 extract.py	 mapper.py  readers.py

../../iomeval/iomeval/__pycache__:
__init__.cpython-312.pyc  downloaders.cpython-312.pyc  readers.cpython-312.pyc
_modidx.cpython-312.pyc   extract.cpython-312.pyc      themes.cpython-312.pyc
core.cpython-312.pyc	  mapper.cpython-312.pyc

../../iomeval/iomeval/files:
prompts  themes

../../iomeval/iomeval/files/prompts:
gcm.md	select_sections.md  srf_ccps.md  srf_enablers.md  srf_outputs.md

../../iomeval/iomeval/files/themes:
crosscutting_priorities.json  gcms_short.md	   srf_outputs_to_gcm.json
gcm_to_srf_outputs.json       srf_enablers.json
gcms_long.md		      srf_objectives.json


In [ ]:
!ls -R ../../iomeval/nbs

../../iomeval/nbs:
00_core.ipynb	      04_themes.ipynb	 files	      sidebar.yml
01_readers.ipynb      05_mapper.ipynb	 index.ipynb  styles.css
02_downloaders.ipynb  06_pipeline.ipynb  nbdev.yml
03_extract.ipynb      _quarto.yml	 ocr_temp

../../iomeval/nbs/files:
glossaries  prompts  test  themes

../../iomeval/nbs/files/glossaries:
migration.json

../../iomeval/nbs/files/prompts:
gcm.md	select_sections.md  srf_ccps.md  srf_enablers.md  srf_outputs.md

../../iomeval/nbs/files/test:
AAP%20Evaluation%20Report_final_		   evaluations.json	pdfs
eval_repo_iom.xlsx				   image_cache.json	results
eval_reports					   iom_evals_test.json
evaluation-search-export-11_13_2025--18_09_44.csv  md

../../iomeval/nbs/files/test/AAP%20Evaluation%20Report_final_:
img	    page_23.md	page_38.md  page_52.md	page_67.md  page_81.md
page_1.md   page_24.md	page_39.md  page_53.md	page_68.md  page_82.md
page_10.md  page_25.md	page_4.md   page_54.md	page_69.md  page_83.md
page_11.md  page_26.md	page_40.md  page_55.

settings.ini

```
[DEFAULT]
repo = iomeval
lib_name = iomeval
version = 0.0.7
min_python = 3.9
license = apache2
black_formatting = False
doc_path = _docs
lib_path = iomeval
nbs_path = nbs
recursive = True
tst_flags = notest
put_version_in_init = True
update_pyproject = True
branch = main
custom_sidebar = True
doc_host = https://franckalbinet.github.io
doc_baseurl = /iomeval
git_url = https://github.com/franckalbinet/iomeval
title = iomeval
audience = Developers
author = Solveit
author_email = nobody@fast.ai
copyright = 2025 onwards, Solveit
description = Streamline evaluation evidence mapping at scale with LLMs
keywords = nbdev jupyter notebook python
language = English
status = 3
user = franckalbinet
requirements = fastcore pandas lisette pydantic tiktoken mistocr python-dotenv
package_data = files/prompts/*.md files/themes/*.json files/themes/*.md
readme_nb = index.ipynb
allowed_metadata_keys = 
allowed_cell_metadata_keys = 
jupyter_hooks = False
clean_ids = True
clear_all = False
cell_number = True
skip_procs = 
```

setup.py
```
from pkg_resources import parse_version
from configparser import ConfigParser
import setuptools, shlex
assert parse_version(setuptools.__version__)>=parse_version('36.2')

# note: all settings are in settings.ini; edit there, not here
config = ConfigParser(delimiters=['='])
config.read('settings.ini', encoding='utf-8')
cfg = config['DEFAULT']

cfg_keys = 'version description keywords author author_email'.split()
expected = cfg_keys + "lib_name user branch license status min_python audience language".split()
for o in expected: assert o in cfg, "missing expected setting: {}".format(o)
setup_cfg = {o:cfg[o] for o in cfg_keys}

licenses = {
    'apache2': ('Apache Software License 2.0','OSI Approved :: Apache Software License'),
    'mit': ('MIT License', 'OSI Approved :: MIT License'),
    'gpl2': ('GNU General Public License v2', 'OSI Approved :: GNU General Public License v2 (GPLv2)'),
    'gpl3': ('GNU General Public License v3', 'OSI Approved :: GNU General Public License v3 (GPLv3)'),
    'bsd3': ('BSD License', 'OSI Approved :: BSD License'),
}
statuses = [ '1 - Planning', '2 - Pre-Alpha', '3 - Alpha',
    '4 - Beta', '5 - Production/Stable', '6 - Mature', '7 - Inactive' ]
py_versions = '3.6 3.7 3.8 3.9 3.10 3.11 3.12'.split()

requirements = shlex.split(cfg.get('requirements', ''))
if cfg.get('pip_requirements'): requirements += shlex.split(cfg.get('pip_requirements', ''))
min_python = cfg['min_python']
lic = licenses.get(cfg['license'].lower(), (cfg['license'], None))
dev_requirements = (cfg.get('dev_requirements') or '').split()

package_data = dict()
pkg_data = cfg.get('package_data', None)
if pkg_data:
    package_data[cfg['lib_name']] =  pkg_data.split() # split as multiple files might be listed
# Add package data to setup_cfg for setuptools.setup(..., **setup_cfg)
setup_cfg['package_data'] = package_data

print(setup_cfg['package_data'])

setuptools.setup(
    name = cfg['lib_name'],
    license = lic[0],
    classifiers = [
        'Development Status :: ' + statuses[int(cfg['status'])],
        'Intended Audience :: ' + cfg['audience'].title(),
        'Natural Language :: ' + cfg['language'].title(),
    ] + ['Programming Language :: Python :: '+o for o in py_versions[py_versions.index(min_python):]] + (['License :: ' + lic[1] ] if lic[1] else []),
    url = cfg['git_url'],
    packages = setuptools.find_packages(),
    include_package_data = True,
    install_requires = requirements,
    extras_require={ 'dev': dev_requirements },
    dependency_links = cfg.get('dep_links','').split(),
    python_requires  = '>=' + cfg['min_python'],
    long_description = open('README.md', encoding='utf-8').read(),
    long_description_content_type = 'text/markdown',
    zip_safe = False,
    entry_points = {
        'console_scripts': cfg.get('console_scripts','').split(),
        'nbdev': [f'{cfg.get("lib_path")}={cfg.get("lib_path")}._modidx:d']
    },
    **setup_cfg)
```

In [ ]:
sut = read_url("https://setuptools.pypa.io/en/latest/userguide/datafiles.html")
sut[:200]

'Contents Menu Expand Light mode Dark mode Auto light/dark, in light mode Auto light/dark, in dark mode Skip to content\n\n[setuptools 80.9.0 documentation](../index.html)\n\n[ ![Logo](https://setuptools.p'

I am struggling with consuming local `files` when iomeval is installed locally (in a non notebook folder). Can you check the doc above to investigate why I consistently getting this error "FileNotFoundError: [Errno 2] No such file or directory: '/Users/franckalbinet/pro/dev/iom-pipeline/.venv/lib/python3.12/site-packages/iomeval/files/prompts/select_sections.md'"

You can access python files through the &`nb_to_md` if required.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

This is a classic package data issue! Let me investigate how your code is trying to load those files. Could you point me to which module is loading these prompt files? I suspect it's in `extract.py` or `mapper.py`.

Let me check the relevant notebook to see how the file paths are being constructed:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01T9rGEB8hrPZZ1R8WRnn6wS",
  "call": {
    "function": "nb_to_md",
    "arguments": {
      "path": "../../iomeval/nbs/05_mapper.ipynb"
    }
  },
  "result": "'# Mapper\\n\\n> Maps IOM evaluation report against evaluation frameworks \\n\\n---\\n\\nMaps evaluation r<TRUNCATED>"
}
```

</details>

Now let me check `core.py` to see how `load_prompt` is implemented:

<details class='tool-usage-details'>

```json
{
  "id": "toolu_01SFSzZs5BBqeQtFC25PMjL5",
  "call": {
    "function": "nb_to_md",
    "arguments": {
      "path": "../../iomeval/nbs/00_core.ipynb"
    }
  },
  "result": "'# Core\\n\\n> Utilities function\\n\\n---\\n\\n```python\\n#| default_exp core\\n```\\n\\n```python\\n#| expor<TRUNCATED>"
}
```

</details>

I found the issue! Look at the `load_prompt` function:

```python
def load_prompt(name:str, path:Path|str|None=None) -> str:
    if path is None:
        try: path = Path(__file__).parent / 'files' / 'prompts'
        except NameError: path = Path('files/prompts')  # notebook fallback
    return (Path(path)/f'{name}.md').read_text()
```

The path resolution uses `Path(__file__).parent / 'files' / 'prompts'`, which should work **if** the files are actually installed in the package.

The problem is likely in your `settings.ini` / `setup.py`. You have:

```
package_data = files/prompts/*.md files/themes/*.json files/themes/*.md
```

But the `files` folder needs to exist **inside** `iomeval/` (the lib_path), not just in `nbs/`. 

Can you confirm: does `../../iomeval/iomeval/files/prompts/select_sections.md` actually exist? From your earlier `ls -R` output, I see:

```
../../iomeval/iomeval/files/prompts:
gcm.md  select_sections.md  srf_ccps.md  srf_enablers.md  srf_outputs.md
```

So the files **do** exist in the source. The issue is they're not being included when you `pip install`. 

Do you have a `MANIFEST.in` file in your iomeval repo root? That's often needed to ensure non-Python files get included in the distribution.

Yes here is the MANIFEST.in
```
include settings.ini
include LICENSE
include CONTRIBUTING.md
include README.md
recursive-exclude * __pycache__
```